# Mesh generation for tutorial 03

In [ ]:
import os
import meshio
import pygmsh
from dolfinx import MPI
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
p0 = geom.add_point([0.0, 0.0, 0.0], lcar)
p1 = geom.add_point([0.0, +r, 0.0], lcar)
p2 = geom.add_point([0.0, -r, 0.0], lcar)
c0 = geom.add_circle_arc(p1, p0, p2)
c1 = geom.add_circle_arc(p2, p0, p1)
l0 = geom.add_line(p2, p1)
geom.add_physical([c0, c1], label=1)
geom.add_physical(l0, label=2)
line_loop_left = geom.add_line_loop([c0, l0])
line_loop_right = geom.add_line_loop([c1, -l0])
semicircle_left = geom.add_plane_surface(line_loop_left)
semicircle_right = geom.add_plane_surface(line_loop_right)
geom.add_physical(semicircle_left, label=11)
geom.add_physical(semicircle_right, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Save temporary mesh and subdomains (pygmsh format)

In [ ]:
meshio.write("circle_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"triangle": pygmsh_mesh.cells_dict["triangle"]},
    cell_data={"subdomains": [pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10]}
))

### Save temporary boundaries (pygmsh format)

In [ ]:
meshio.write("circle_boundaries_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"line": pygmsh_mesh.cells_dict["line"]},
    cell_data={"boundaries": [pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]]}
))

### Read back in mesh (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "circle_tmp.xdmf", "r") as infile:
    mesh = infile.read_mesh(name="Grid")
    mesh.name = "mesh"

In [ ]:
plot(mesh)

### Read back in subdomains and boundaries (dolfinx format)

In [ ]:
mesh.create_connectivity_all()
with XDMFFile(MPI.comm_world, "circle_tmp.xdmf", "r") as infile:
    subdomains = infile.read_meshtags(mesh, name="Grid")
    subdomains.name = "subdomains"
with XDMFFile(MPI.comm_world, "circle_boundaries_tmp.xdmf", "r") as infile:
    boundaries = infile.read_meshtags(mesh, name="Grid")
    boundaries.name = "boundaries"

### Save final mesh, subdomains and boundaries (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "circle.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    file.write_meshtags(boundaries)

In [ ]:
os.remove("circle_tmp.xdmf")
os.remove("circle_tmp.h5")
os.remove("circle_boundaries_tmp.xdmf")
os.remove("circle_boundaries_tmp.h5")